In [1]:
import numpy as np
from datetime import datetime

# 수치미분 함수

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

# sigmoid 함수

def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [2]:
# LogicGate Class

class LogicGate:
    
    # 생성자
    # xdata, tdata => numpy.array(...)
    def __init__(self, gate_name, input_nodes, hidden_1_nodes, hidden_2_nodes, output_nodes, learning_rate):
        
        self.name = gate_name
                
        # 2층 hidden1 layer 
        # 가중치 W, 바이어스 b 초기화
        self.W2 = np.random.rand(input_nodes, hidden_1_nodes)  
        self.b2 = np.random.rand(hidden_1_nodes)
        
        # 3층 hidden2 layer 
        self.W3 = np.random.rand(hidden_1_nodes, hidden_2_nodes)
        self.b3 = np.random.rand(hidden_2_nodes)
                        
        # 4층 output layer 
        self.W4 = np.random.rand(hidden_2_nodes, output_nodes)
        self.b4 = np.random.rand(output_nodes)
        
        # 학습률 learning rate 초기화
        self.learning_rate = learning_rate
            
        print(self.name + " object is created")
        
    # 손실함수
    def feed_forward(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z1 = np.dot(self.input_data, self.W2) + self.b2
        y1 = sigmoid(z1)
        
        z2 = np.dot(y1, self.W3) + self.b3
        y2 = sigmoid(z2)
        
        z3 = np.dot(y2, self.W4) + self.b4
        y = sigmoid(z3)
    
        # cross-entropy 
        return  -np.sum( self.target_data*np.log(y + delta) + (1-self.target_data)*np.log((1 - y)+delta ) )
    
    # obtain W and b
    def get_W_b(self):
        
        return self.W2,  self.b2, self.W3, self.b3
    
    # 손실 값 계산
    def loss_val(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z1 = np.dot(self.input_data, self.W2) + self.b2
        y1 = sigmoid(z1)
        
        z2 = np.dot(y1, self.W3) + self.b3
        y2 = sigmoid(z2)
        
        z3 = np.dot(y2, self.W4) + self.b4
        y = sigmoid(z3)
    
        # cross-entropy 
        return  -np.sum( self.target_data*np.log(y + delta) + (1-self.target_data)*np.log((1 - y)+delta ) )
    
    # query, 즉 미래 값 예측 함수
    def predict(self, input_data):    
        
        z1 = np.dot(input_data, self.W2) + self.b2
        y1 = sigmoid(z1)
        
        z2 = np.dot(y1, self.W3) + self.b3
        y2 = sigmoid(z2)
        
        z3 = np.dot(y2, self.W4) + self.b4
        y = sigmoid(z3)
    
        if y >= 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result

    # accuracy method 1
    def accuracy1(self, input_data, target_data):
        
        matched_list = []
        not_matched_list = []
        
        for index in range(len(input_data)):
            
            (real_val, logical_val) = self.predict(input_data[index])
            
            if logical_val == target_data[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        accuracy_result = len(matched_list) / len(input_data)
        
        print("Accuracy => ", accuracy_result)
        
        return matched_list, not_matched_list
    
    # accuracy method 2
    def accuracy2(self, test_data):
        
        matched_list = []
        not_matched_list = []
        
        input_data = test_data[ :, 0:-1]
        target_data = test_data[ :, -1]
        
        for index in range(len(input_data)):
            
            (real_val, logical_val) = self.predict(input_data[index])
            
            if logical_val == target_data[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        accuracy_result = len(matched_list) / len(input_data)
        
        print("Accuracy => ", accuracy_result)
        
        return matched_list, not_matched_list
    
        
    # 수치미분을 이용하여 손실함수가 최소가 될때 까지 학습하는 함수
    def train(self, input_data, target_data):
        
        self.input_data = input_data
        self.target_data = target_data
        
        f = lambda x : self.feed_forward()
        
        self.W2 -= self.learning_rate * numerical_derivative(f, self.W2)
    
        self.b2 -= self.learning_rate * numerical_derivative(f, self.b2)
        
        self.W3 -= self.learning_rate * numerical_derivative(f, self.W3)
    
        self.b3 -= self.learning_rate * numerical_derivative(f, self.b3)
        
        self.W4 -= self.learning_rate * numerical_derivative(f, self.W4)
    
        self.b4 -= self.learning_rate * numerical_derivative(f, self.b4)

In [3]:
and_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
and_tdata = np.array([0, 0, 0, 1])

# hyper-parameter
i_nodes = 2   # input nodes
h1_nodes = 8  # hidden 1 nodes
h2_nodes = 4  # hidden 2 nodes
o_nodes = 1   # output nodes
iteration_count = 10001  # iteration count
lr = 1e-1  # learning rate, 1e-2 로 하면 손실함수가 감소하지 않음

# AND Gate 객체 생성 및 training
and_obj = LogicGate("AND", i_nodes, h1_nodes, h2_nodes, o_nodes, lr)

start_time = datetime.now()

for step in range(iteration_count):

    and_obj.train(and_xdata.reshape(4,2), and_tdata.reshape(4,1))
    
    if (step % 500 == 0):
        print("step = ", step, "loss value = ", and_obj.loss_val())
              
end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

AND object is created
step =  0 loss value =  5.316789995766165
step =  500 loss value =  2.250566733576382
step =  1000 loss value =  2.229302707784173
step =  1500 loss value =  1.4814302809504913
step =  2000 loss value =  0.10015948486839496
step =  2500 loss value =  0.04477939006096085
step =  3000 loss value =  0.028690982875519615
step =  3500 loss value =  0.02106194757078112
step =  4000 loss value =  0.016610819343539064
step =  4500 loss value =  0.013694417976227218
step =  5000 loss value =  0.011635937002425321
step =  5500 loss value =  0.010105605161474112
step =  6000 loss value =  0.008923504527092332
step =  6500 loss value =  0.007983117765169672
step =  7000 loss value =  0.007217352780670541
step =  7500 loss value =  0.006581843019813169
step =  8000 loss value =  0.006046074771443975
step =  8500 loss value =  0.005588371150691669
step =  9000 loss value =  0.005192908812646464
step =  9500 loss value =  0.004847866997766953
step =  10000 loss value =  0.004544

In [4]:
# AND Gate prediction
print(and_obj.name, "\n")

test_data = np.array([ [0, 0, 0], [0, 1, 0], [1, 0, 0], [1, 1, 1] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 0, 0, 1])

(true_list, false_list) = and_obj.accuracy1(test_xdata.reshape(4,2), test_tdata.reshape(4,1))
(true_list, false_list) = and_obj.accuracy2(test_data)

AND 

Accuracy =>  1.0
Accuracy =>  1.0


In [5]:
or_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
or_tdata = np.array([0, 1, 1, 1])

# hyper-parameter
i_nodes = 2   # input nodes
h1_nodes = 8  # hidden 1 nodes
h2_nodes = 4  # hidden 2 nodes
o_nodes = 1   # output nodes
iteration_count = 10001  # iteration count
lr = 1e-1  # learning rate, 1e-2 로 하면 손실함수가 감소하지 않음

# OR Gate 객체 생성 및 training
or_obj = LogicGate("OR", i_nodes, h1_nodes, h2_nodes, o_nodes, lr)

start_time = datetime.now()

for step in range(iteration_count):

    or_obj.train(or_xdata.reshape(4,2), or_tdata.reshape(4,1))
    
    if (step % 500 == 0):
        print("step = ", step, "loss value = ", or_obj.loss_val())
              
end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

OR object is created
step =  0 loss value =  2.687011359714549
step =  500 loss value =  1.155713857797883
step =  1000 loss value =  0.09275562698218025
step =  1500 loss value =  0.04415121614501615
step =  2000 loss value =  0.028623492198114435
step =  2500 loss value =  0.02102796413954626
step =  3000 loss value =  0.01653654252492502
step =  3500 loss value =  0.013575755557700226
step =  4000 loss value =  0.01148089587940022
step =  4500 loss value =  0.009922981530930216
step =  5000 loss value =  0.008720662178320956
step =  5500 loss value =  0.0077658098889945935
step =  6000 loss value =  0.006989980319817092
step =  6500 loss value =  0.006347755086744255
step =  7000 loss value =  0.0058078214531895375
step =  7500 loss value =  0.005347898641464387
step =  8000 loss value =  0.00495170582462636
step =  8500 loss value =  0.004607073816264945
step =  9000 loss value =  0.004304727088640166
step =  9500 loss value =  0.004037474118708988
step =  10000 loss value =  0.003

In [6]:
# OR Gate prediction
print(or_obj.name, "\n")

test_data = np.array([ [0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 1] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 1, 1, 1])

(true_list, false_list) = or_obj.accuracy1(test_xdata.reshape(4,2), test_tdata.reshape(4,1))
(true_list, false_list) = or_obj.accuracy2(test_data)

OR 

Accuracy =>  1.0
Accuracy =>  1.0


In [7]:
nand_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
nand_tdata = np.array([1, 1, 1, 0])

# hyper-parameter
i_nodes = 2   # input nodes
h1_nodes = 8  # hidden 1 nodes
h2_nodes = 4  # hidden 2 nodes
o_nodes = 1   # output nodes
iteration_count = 10001  # iteration count
lr = 1e-1  # learning rate, 1e-2 로 하면 손실함수가 감소하지 않음

# OR Gate 객체 생성 및 training
nand_obj = LogicGate("NAND", i_nodes, h1_nodes, h2_nodes, o_nodes, lr)

start_time = datetime.now()

for step in range(iteration_count):

    nand_obj.train(nand_xdata.reshape(4,2), nand_tdata.reshape(4,1))
    
    if (step % 500 == 0):
        print("step = ", step, "loss value = ", nand_obj.loss_val())
              
end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

NAND object is created
step =  0 loss value =  2.996916743838344
step =  500 loss value =  2.2526200770855103
step =  1000 loss value =  2.240258529300193
step =  1500 loss value =  1.7028007673228334
step =  2000 loss value =  0.10571869149149084
step =  2500 loss value =  0.046408316863294725
step =  3000 loss value =  0.02966302768412992
step =  3500 loss value =  0.02180373372911632
step =  4000 loss value =  0.01724263556686937
step =  4500 loss value =  0.014263345977660009
step =  5000 loss value =  0.01216422262161573
step =  5500 loss value =  0.010605165259312137
step =  6000 loss value =  0.009401336658161666
step =  6500 loss value =  0.008443635092110682
step =  7000 loss value =  0.007663504602039297
step =  7500 loss value =  0.00701569796379888
step =  8000 loss value =  0.006469144587482439
step =  8500 loss value =  0.006001800277782923
step =  9000 loss value =  0.005597590432692678
step =  9500 loss value =  0.005244516856911777
step =  10000 loss value =  0.0049334

In [8]:
# NAND Gate prediction
print(nand_obj.name, "\n")

test_data = np.array([ [0, 0, 1], [0, 1, 1], [1, 0, 1], [1, 1, 0] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 1, 1, 1, 0])

(true_list, false_list) = nand_obj.accuracy1(test_xdata.reshape(4,2), test_tdata.reshape(4,1))
(true_list, false_list) = nand_obj.accuracy2(test_data)

NAND 

Accuracy =>  1.0
Accuracy =>  1.0


In [9]:
xor_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
xor_tdata = np.array([0, 1, 1, 0])

# hyper-parameter
i_nodes = 2   # input nodes
h1_nodes = 8  # hidden 1 nodes
h2_nodes = 4  # hidden 2 nodes
o_nodes = 1   # output nodes
iteration_count = 10001  # iteration count
lr = 1e-1  # learning rate, 1e-2 로 하면 손실함수가 감소하지 않음

# XOR Gate 객체 생성 및 training
xor_obj = LogicGate("NAND", i_nodes, h1_nodes, h2_nodes, o_nodes, lr)

start_time = datetime.now()

for step in range(iteration_count):

    xor_obj.train(xor_xdata.reshape(4,2), xor_tdata.reshape(4,1))
    
    if (step % 500 == 0):
        print("step = ", step, "loss value = ", xor_obj.loss_val())
              
end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

NAND object is created
step =  0 loss value =  3.917622408978869
step =  500 loss value =  2.7717474363931522
step =  1000 loss value =  2.771021930775164
step =  1500 loss value =  2.769771433335058
step =  2000 loss value =  2.7669968307991235
step =  2500 loss value =  2.757558771307121
step =  3000 loss value =  2.669793181243333
step =  3500 loss value =  2.0709855200828775
step =  4000 loss value =  1.878140843537905
step =  4500 loss value =  0.24349002943957543
step =  5000 loss value =  0.08689583057335712
step =  5500 loss value =  0.049273940215170495
step =  6000 loss value =  0.03234272128975218
step =  6500 loss value =  0.023294546449827102
step =  7000 loss value =  0.017919570912346403
step =  7500 loss value =  0.014438232263165414
step =  8000 loss value =  0.012026732216535756
step =  8500 loss value =  0.01026859976210201
step =  9000 loss value =  0.008935287216490008
step =  9500 loss value =  0.00789233663469464
step =  10000 loss value =  0.007055977355855739



In [10]:
# XOR Gate prediction
print(xor_obj.name, "\n")

test_data = np.array([ [0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 0] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 1, 1, 0])

(true_list, false_list) = xor_obj.accuracy1(test_xdata.reshape(4,2), test_tdata.reshape(4,1))
(true_list, false_list) = xor_obj.accuracy2(test_data)

NAND 

Accuracy =>  1.0
Accuracy =>  1.0
